In [16]:
import numpy as np
import random
import math
import gc
from random import sample
from time import time
from tabulate import tabulate

#Graph Generator Function

In [17]:
def generate_graph(v, e, unconnected):
	for i in range(100): # prevent infinite loop
		edges = []
		for i in range(e):
			conflict = True
			while (conflict):
				edge = sample([j for j in range(v - unconnected)], 2)
				if (min(edge), max(edge)) not in edges:
					conflict = False
					edges.append((min(edge), max(edge)))
		edges = sorted(edges, key=lambda row: (row[0], row[1]))
		flat_list = [i for edge in edges for i in edge]

		for i in range(v - unconnected):
			if i not in flat_list:
				break
		else:
			return edges
	raise SystemExit("Unable to generate graph.\nPlease change the parameter")

#Exact Method Configuration

In [18]:
def isCover(V, k, E):
	Set = (1 << k) - 1
	limit = (1 << V)
	vis = [[None] * V for i in range(V)]
	while (Set < limit):
		vis = [[0] * V for i in range(V)]
		cnt = 0
		j = 1
		v = 1
		while(j < limit):
			if (Set & j and v < V and k < V):
				for k in range(0, V):
					if (gr[v][k] and not vis[v][k]):
						vis[v][k] = 1
						vis[k][v] = 1
						cnt += 1
			j = j << 1
			v += 1
		if (cnt == E):
			return True
		c = Set & -Set
		r = Set + c
		Set = (((r ^ Set) >> 2) // c) | r
	return False

In [19]:
def findMinCover(n, m):
	left = 1
	right = n
	while (right > left):
		mid = (left + right) >> 1
		if (isCover(n, mid, m) == False):
			left = mid + 1
		else:
			right = mid
	return left

In [20]:
def insertEdge(u, v):
	gr[u][v] = 1
	gr[v][u] = 1

#Non-Exact Method Configuration

In [21]:
def chromosomes_gen(n,k,pop_init):
    lst = []
    for i in range(pop_init):
        chromosome = np.zeros(n,dtype= int)
        samples = random.sample(range(0,n), k = k)
        for j in range(k):
            chromosome[samples[j]] = 1
        lst.append(chromosome)
    return lst

In [22]:
def cost(cmbn,n,edges):
    obstacles = 0
    for e in edges:
        (u,v) = e
        if ((cmbn[u]==0) & (cmbn[v]==0)):
            obstacles += 1
    return obstacles

In [23]:
def selection(lst,pop_total,n,edges):
    score = []
    output_lst = []
    len_lst = len(lst)
    for i in range(len_lst):
        score.append(cost(lst[i],n,edges))
    sorted_index = np.argsort(score)
    cnt = 0
    for i in range(len_lst):
        output_lst.append(lst[sorted_index[i]])
        if((i+1) == pop_total):
            break
    lst = output_lst
    return lst,score[sorted_index[0]]

In [24]:
mutat_prob = 0.05
def cross_over_mutate_extended(lst,n,k,mutat_prob,pop_total,edges):
    new_lst = lst.copy()
    len_lst = len(lst)
    cross_over_prob = 0.50
    variations = 1
    
    #Crossover
    for i in range(len_lst):
        for v in range(variations):
            tmp = lst[i].copy()

            mate_with = lst[int(random.uniform(0,len_lst))]

            tmp_unique = []
            mate_with_unique = []

            for j in range(n):
                if(tmp[j]==1):
                    tmp_unique.append(j)
                if(mate_with[j]==1):
                    mate_with_unique.append(j)

            tmp_unique = np.setdiff1d(tmp,mate_with)
            random.shuffle(tmp_unique)
            mate_with_unique = np.setdiff1d(mate_with,tmp)
            random.shuffle(mate_with_unique)

            swap = math.ceil(cross_over_prob * min(len(tmp_unique),len(mate_with_unique)))

            for j in range(swap):
                tmp[mate_with_unique[j]] = 1
                tmp[tmp_unique[j]] = 0

            # Mutation 
            zeroes = []
            ones = []
            for j in range(n):
                if tmp[j]==1:
                    ones.append(j)
                else:
                    zeroes.append(j)
            
            random.shuffle(ones)
            random.shuffle(zeroes)

            coin_toss = random.random()
            if(coin_toss <= 0.5):
                swaps = min(len(ones),len(zeroes))

                for j in range(swaps):
                    coin_toss2 = random.random()
                    if(coin_toss2 < mutat_prob):
                        tmp[ones[j]] = 0
                        tmp[zeroes[j]] = 1
                        #Swapping logic
                        dummy = ones[j]
                        ones[j] = zeroes[j]
                        zeroes[j] = dummy
            else:    
                mutate_lst = []
                for e in edges:
                    (u,v) = e
                    if((tmp[u] == 0) & (tmp[v] == 0)):
                        coin_toss2 = random.random()
                        if(coin_toss2 < mutat_prob):
                            coin_toss3 = random.random()
                            if(coin_toss3 <= 0.5):
                                if(u not in mutate_lst):
                                    mutate_lst.append(u)
                            else:
                                if(v not in mutate_lst):
                                    mutate_lst.append(v)

                random.shuffle(mutate_lst)
                sz = min(len(ones),len(mutate_lst))
                
                for j in range(sz):
                    tmp[ones[j]] = 0
                    tmp[mutate_lst[j]] = 1
                    #Swapping logic
                    dummy = ones[j]
                    ones[j] = mutate_lst[j]
                    mutate_lst[j] = dummy
                
            new_lst.append(tmp)
    
    return new_lst

In [25]:
def environment(n,k,mutat_prob,pop_init,pop_total,max_iterate,edges):
    lst = chromosomes_gen(n,k,pop_init)
    for it in range(max_iterate):
        lst = cross_over_mutate_extended(lst,n,k,mutat_prob,pop_total,edges)
        lst,cost_value = selection(lst,pop_total,n,edges)
        # if (it%10)==9:
        #     print("k = {}, Iteration = {}, Cost = {}".format(k,it+1,cost_value))
        if cost_value==0:
            break
    result = []
    soln = lst[0]
    for j in range(len(soln)):
        if(soln[j] == 1):
            result.append(j)
    # print("k = {}, Iteration = {}, Cost = {}\nVertex Cover = {}".format(k,it,cost_value,result))
    return cost_value,result

In [26]:
def free_memory():
    gc.collect()

In [27]:
def mfind(n,mutat_prob,pop_init,pop_total,max_iterate,edges,start,end):
    result_dict = {}
    l = start
    h = end
    ans = 0
    while(l<=h):
        m = int((l+h)/2.0)
        cost_value,result = environment(n,m,mutat_prob,pop_init,pop_total,max_iterate,edges)
        if(cost_value==0):
            result_dict[m] = result
            h = m-1
        else:
            l = m + 1
    return result_dict

#Exact Method Function

In [28]:
def exhaustive_algorithm(nodes, edges):
	global gr
	gr = [[None] * nodes for i in range(nodes)]

	for x, y in edges:
		insertEdge(x, y)
		
	time_now = time()
	vertices = findMinCover(nodes, len(edges))
	time_end = time()
	return (vertices, "{:.4f}s".format(time_end - time_now))

#Non-Exact Method Function

In [29]:
def genetic_algorithm(nodes, edges):
    adjacency_matrix = np.zeros((nodes,nodes),dtype = np.int)
    adjacency_matrix.shape
    for i, j in edges:
        adjacency_matrix[i, j] = 1
    #Vertex Cover Greedy Algorithm
    visited = np.zeros(nodes)
    cnt = 0
    for e in edges:
        (u,v) = e
        if ((visited[u]==0) & (visited[v]==0)):
            visited[u] = 1
            visited[v] = 1
            cnt+=2
    approximation_algo_result = cnt
    n = nodes
    pop_total = int(50 * max(1, round(n / 5.0))) # max population allowed in the environment
    pop_init = int(20 * max(1, round(n / 5.0)))
    max_iterate = int(7 * max(1, round(n / 5.0)))

    for i, j in edges:
        adjacency_matrix[i, j] = 1

    time_now = time()
    result = mfind(n, mutat_prob, pop_init, pop_total, max_iterate, edges, int(approximation_algo_result / 2), n)
    time_end = time()
    vertices = len(result[list(result.keys())[-1]])
    return (vertices, "{:.4f}s".format(time_end - time_now))

##Experiment 1 (Changes on vertices)

In [19]:
edge = 30
unconnected = 0
instance = 5

header = ["#", "Vertices", "Edges", "Unconnected Vertices", "Exhaustive Time Taken", "Genetic Time Taken", "Genetic Accuracy"]
data = []

for i in range(13):
	vertices = 10 + i
	for j in range(1, instance + 1):
		edges = generate_graph(vertices, edge, unconnected)
		exhaustive_vc, exhaustive = exhaustive_algorithm(vertices, edges)
		genetic_vc, genetic = genetic_algorithm(vertices, edges)

		row = [i * instance + j, vertices, edge, unconnected, exhaustive, genetic, exhaustive_vc == genetic_vc]
		data.append(row)
print("Experiment 1 (Changes on vertices)")
print(tabulate(data, headers=header, tablefmt="pretty", disable_numparse=True, stralign="right"))

Experiment 1 (Changes on vertices)
+----+----------+-------+----------------------+-----------------------+--------------------+------------------+
| #  | Vertices | Edges | Unconnected Vertices | Exhaustive Time Taken | Genetic Time Taken | Genetic Accuracy |
+----+----------+-------+----------------------+-----------------------+--------------------+------------------+
| 1  |    10    |  30   |          0           |        0.0080s        |      0.6173s       |       True       |
| 2  |    10    |  30   |          0           |        0.0099s        |      0.5027s       |       True       |
| 3  |    10    |  30   |          0           |        0.0070s        |      0.4697s       |      False       |
| 4  |    10    |  30   |          0           |        0.0130s        |      0.5495s       |       True       |
| 5  |    10    |  30   |          0           |        0.0070s        |      0.5936s       |      False       |
| 6  |    11    |  30   |          0           |        0.013

##Experiment 2 (Changes on edges)

In [30]:
vertices = 20
unconnected = 0
instance = 5

header = ["#", "Vertices", "Edges", "Unconnected Vertices", "Exhaustive Time Taken", "Genetic Time Taken", "Genetic Accuracy"]
data = []

for i in range(13):
	edge = 30 + i * 3
	for j in range(1, instance + 1):
		edges = generate_graph(vertices, edge, unconnected)
		exhaustive_vc, exhaustive = exhaustive_algorithm(vertices, edges)
		genetic_vc, genetic = genetic_algorithm(vertices, edges)

		row = [i * instance + j, vertices, edge, unconnected, exhaustive, genetic, exhaustive_vc == genetic_vc]
		data.append(row)
print("Experiment 2 (Changes on edges)")
print(tabulate(data, headers=header, tablefmt="pretty", disable_numparse=True, stralign="right"))

Experiment 2 (Changes on edges)
+----+----------+-------+----------------------+-----------------------+--------------------+------------------+
| #  | Vertices | Edges | Unconnected Vertices | Exhaustive Time Taken | Genetic Time Taken | Genetic Accuracy |
+----+----------+-------+----------------------+-----------------------+--------------------+------------------+
| 1  |    20    |  30   |          0           |       11.5122s        |      4.2038s       |      False       |
| 2  |    20    |  30   |          0           |       15.2861s        |      4.0113s       |       True       |
| 3  |    20    |  30   |          0           |       10.9168s        |      4.4002s       |      False       |
| 4  |    20    |  30   |          0           |       14.5700s        |      2.9252s       |       True       |
| 5  |    20    |  30   |          0           |       18.6790s        |      2.7905s       |       True       |
| 6  |    20    |  33   |          0           |       15.6661s 

##Experiment 3 (Changes on unconnected vertices)

In [31]:
vertices = 20
edge = 30
instance = 5

header = ["#", "Vertices", "Edges", "Unconnected Vertices", "Exhaustive Time Taken", "Genetic Time Taken", "Genetic Accuracy"]
data = []

for i in range(10):
	unconnected = i + 1
	for j in range(1, instance + 1):
		edges = generate_graph(vertices, edge, unconnected)
		exhaustive_vc, exhaustive = exhaustive_algorithm(vertices, edges)
		genetic_vc, genetic = genetic_algorithm(vertices, edges)

		row = [i * instance + j, vertices, edge, unconnected, exhaustive, genetic, exhaustive_vc == genetic_vc]
		data.append(row)
print("Experiment 3 (Changes on unconnected vertices)")
print(tabulate(data, headers=header, tablefmt="pretty", disable_numparse=True, stralign="right"))

Experiment 3 (Changes on unconnected vertices)
+----+----------+-------+----------------------+-----------------------+--------------------+------------------+
| #  | Vertices | Edges | Unconnected Vertices | Exhaustive Time Taken | Genetic Time Taken | Genetic Accuracy |
+----+----------+-------+----------------------+-----------------------+--------------------+------------------+
| 1  |    20    |  30   |          1           |       14.4005s        |      2.5282s       |       True       |
| 2  |    20    |  30   |          1           |       13.0491s        |      2.3836s       |       True       |
| 3  |    20    |  30   |          1           |       10.3693s        |      2.4405s       |      False       |
| 4  |    20    |  30   |          1           |       12.7289s        |      2.2091s       |      False       |
| 5  |    20    |  30   |          1           |       12.8855s        |      2.8105s       |       True       |
| 6  |    20    |  30   |          2           | 